# GBM-LCCM Example
The purpose of this notebook is to demonstrate how the Gaussian Bernoulli - Latent Class Choice Model (GBM-LCCM) code from Sfeir et al. (2020) could be used.

The application is based on the "London Dataset" from Hillel et al.(2018) which is available online as a supplementary material to the paper.

The dataset contains individual trip diaries of the London Travel Demand Survey (LTDS) from April 2012 to March 2015 with their corresponding modes alternatives extracted from a Google directions application programming interface (API) and corresponding estimates of car operating costs and public transport fares. The dataset consists of 81,086 trips, four modes (walke, cycle, public transport, and drive), and different trip purposes (e.g. Home-Based Work, Home-Based Education, etc.).

The required steps to build a travel mode choie model and run the GBM-LCCM code successfully are listed below.

##### References
G. Sfeir, M. Abou-Zeid, F. Rodrigues, F. C. Pereira, and I. Kaysi, “Semi-nonparametric Latent Class Choice Model with a Flexible Class Membership Component: A Mixture Model Approach,” arXiv Prepr. arXiv2007.02739, 2020.

T. Hillel, M.Z.E.B., Elshafie, and Y. Jin, "Recreating passenger mode choice-sets for transport
simulation: A case study of London, UK," in: Proceedings of the Institution of Civil Engineers - Smart Infrastructure and Construction, 2018

### Step 1: Import the GBM-LCCM Package and All other Necessary Packages

In [1]:
import GBM_LCCM
import numpy as np
import pandas as pd
import pylogit as pl
import warnings
from collections import OrderedDict
from sklearn import preprocessing

### Step 2: Load the Dataset
The GBM-LCCM consists of two sub-models, a class memberhsip model and a class-specific choice model:

- The class membership model is defined as a Gaussian Mixture Model to cluster individuals probabilistically to K clusters (latent classes)

- Conditioned on the class membership of an individual, the class-specific choice model estimates the probability of choosing a specific alternative

The class-specific choice model needs the data to be in "long" format while the class membership model requires a "wide" format.

For more details regarding how to convert your data from wide to long format, you can refer to the following link: https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb

In [2]:
# load the "london" dataset, which is available in wide format
wide_london = pd.read_csv('dataset.csv')

# the .copy() returns a copy of the dataset and ensures that modifications are made to a copy and not the original data
wide_london = wide_london.copy()

In [3]:
wide_london

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_int_total,dur_pt_int_waiting,dur_pt_int_walking,pt_n_interchanges,dur_driving,cost_transit,cost_driving_total,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,0,0,0,0,drive,HBO,Petrol_Car,full,1.0,1,...,0.000000,0.000000,0.000000,0,0.052222,1.5,0.14,0.14,0.0,0.111702
1,1,0,0,1,drive,HBO,Petrol_Car,full,1.0,1,...,0.000000,0.000000,0.000000,0,0.059444,1.5,0.15,0.15,0.0,0.112150
2,2,0,0,2,drive,HBO,Petrol_Car,full,1.0,1,...,0.000000,0.000000,0.000000,0,0.236667,1.5,0.79,0.79,0.0,0.203052
3,3,0,0,3,drive,HBO,Petrol_Car,full,1.0,1,...,0.000000,0.000000,0.000000,0,0.233333,1.5,0.78,0.78,0.0,0.160714
4,4,0,1,2,drive,HBO,Petrol_Car,dis,1.0,1,...,0.000000,0.000000,0.000000,0,0.229167,1.5,0.78,0.78,0.0,0.130909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81081,81081,17615,0,0,drive,HBO,Average_Car,full,1.0,3,...,0.216667,0.197222,0.019444,2,0.859722,4.3,2.48,2.48,0.0,0.402262
81082,81082,17615,0,2,drive,NHBO,Average_Car,full,1.0,3,...,0.183333,0.160278,0.023056,2,0.925833,4.3,2.53,2.53,0.0,0.503750
81083,81083,17615,0,3,drive,HBO,Average_Car,full,1.0,3,...,0.000000,0.000000,0.000000,0,0.112500,1.5,0.32,0.32,0.0,0.234568
81084,81084,17615,1,0,pt,HBW,Average_Car,full,1.0,3,...,0.250000,0.230556,0.019444,2,1.121944,4.4,12.88,2.38,10.5,0.760832


In [4]:
# for this application we will only use Home-Based Work (HBW) trips
wide_london = wide_london[wide_london['purpose']== "HBW"]

# reset index 
wide_london.reset_index(inplace=True, drop=True)

In [5]:
# Note that 5 columns need to be added to the original dataset:
# - travel_mode_nb: 1 for walk, 2 for cycle, 3 for pt, and 4 for drive
# - walk_av, cycle_av, pt_av, drive_av: to determine the availability of each mode (1 if available and 0 otherwise).
# For simplicity, we assume that all modes are available for all individuals

# Define mapping
mode_mapping = {'walk': 1, 'cycle': 2, 'pt': 3, 'drive': 4}

# Apply mapping
wide_london['travel_mode_nb'] = wide_london['travel_mode'].map(mode_mapping)

wide_london['walk_av'] = 1
wide_london['cycle_av'] = 1
wide_london['pt_av'] = 1
wide_london['drive_av'] = 1

/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1317594267.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_london['travel_mode_nb'] = wide_london['travel_mode'].map(mode_mapping)
/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1317594267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_london['walk_av'] = 1
/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1317594267.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [6]:
# look at the first 5 rows of the data
wide_london.head(5).T

,0,1,2,3,4
trip_id,17,125,139,140,141
household_id,3,26,29,29,29
person_n,1,0,0,0,1
trip_n,1,6,0,1,0
travel_mode,pt,drive,pt,pt,pt
purpose,HBW,HBW,HBW,HBW,HBW
fueltype,Average_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car
faretype,full,free,full,full,full
bus_scale,1.0,0.0,1.0,1.0,1.0
survey_year,1,1,1,1,1


### Step 3: Convert the data from wide to long format
In this section, we will use the pylogit package, which is designed for estimating various discrete choice models, including multinomial logit, mixed logit, and nested logit https://pypi.org/project/pylogit/. Additionally, the package can be used to transform data from wide to long format.

In [7]:
# look at the columns of the london dataset
wide_london.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time_linear', 'age', 'female', 'driving_license',
       'car_ownership', 'distance', 'dur_walking', 'dur_cycling',
       'dur_pt_total', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus',
       'dur_pt_int_total', 'dur_pt_int_waiting', 'dur_pt_int_walking',
       'pt_n_interchanges', 'dur_driving', 'cost_transit',
       'cost_driving_total', 'cost_driving_fuel', 'cost_driving_con_charge',
       'driving_traffic_percent', 'travel_mode_nb', 'walk_av', 'cycle_av',
       'pt_av', 'drive_av'],
      dtype='object')

In [8]:
# Create the list of individual specific variables (variables that are specific to an individual 
# and do not vary across alternatives)
ind_variables = wide_london.columns.tolist()[:20]


# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'dur_walking'),
                                               (2, 'dur_cycling'),
                                               (3, 'dur_pt_total'),
                                               (4, 'dur_driving')]),
                        u'travel_cost': dict([(3, 'cost_transit'),
                                              (4, 'cost_driving_total')])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'walk_av',
                          2: 'cycle_av',
                          3: 'pt_av',
                          4: 'drive_av'}


# Determine the columns for: alternative ids, the observation ids and the choice
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
wide_london[obs_id_column] = np.arange(wide_london.shape[0],
                                            dtype=int) + 1


# Create a variable recording the choice column
choice_column = "travel_mode_nb"

/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/2968110328.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_london[obs_id_column] = np.arange(wide_london.shape[0],


In [9]:
# Perform the conversion to long-format
long_london = pl.convert_wide_to_long(wide_london, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_london.head(10).T

/Users/GEOSAF/Desktop/BiogemeEnv/lib/python3.12/site-packages/pylogit/choice_tools.py:454: UserWarning: Note, there are 42 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 32 variables.
  warnings.warn(msg.format(num_dataframe_vars, data_title) +


,0,1,2,3,4,5,6,7,8,9
custom_id,1,1,1,1,2,2,2,2,3,3
mode_id,1,2,3,4,1,2,3,4,1,2
travel_mode_nb,0,0,1,0,0,0,0,1,0,0
trip_id,17,17,17,17,125,125,125,125,139,139
household_id,3,3,3,3,26,26,26,26,29,29
person_n,1,1,1,1,0,0,0,0,0,0
trip_n,1,1,1,1,6,6,6,6,0,0
travel_mode,pt,pt,pt,pt,drive,drive,drive,drive,pt,pt
purpose,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW
fueltype,Average_Car,Average_Car,Average_Car,Average_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car


### Step 4: Model Specification

#### Step 4.1: Class Membership Model

In [10]:
# set the number of clusters (latent classes) to 2
n_classes = 2

In [11]:
# create dummy variables for car ownership
wide_london['car_ownership0'] = (wide_london['car_ownership'] == 0).astype(int)
wide_london['car_ownership1'] = (wide_london['car_ownership'] == 1).astype(int)
wide_london['car_ownership2'] = (wide_london['car_ownership'] == 2).astype(int)

/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1612562048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_london['car_ownership0'] = (wide_london['car_ownership'] == 0).astype(int)
/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1612562048.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_london['car_ownership1'] = (wide_london['car_ownership'] == 1).astype(int)
/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_4673/1612562048.py:4: SettingWithCopyWa

In [12]:
# the class membership model is defined as a Gaussian Bernoulli Mixture Model
# Gaussian Mixture Model is used for continuous variables while Bernoulli Mixture Model for discrete variables

# select the continuous variables for the class membership model
X_Mem_C = wide_london[['age']]
# it is a good practice to standardize the continuous variables for the Gaussian Mixture Model
Standardized_X_Mem_C = preprocessing.scale(X_Mem_C)

# select the discrete variables for the class membership model
X_Mem_D = wide_london[['female','driving_license','car_ownership1','car_ownership2']]
X_Mem_D = X_Mem_D.values

#### Step 4.2: Class-Specific Choice Model
This section is based on the latent class choice model (lccm) package. For more details on how to specify the class-specific choice model, you can check the lccm package on the following link: https://github.com/ferasz/LCCM

You can use alternative specific or generic speficifation for the parameters. 
We will use in this example both types to demonstrate how each one can be used:

- intercept: alternative specific constants included in all the utilities except the one related to walk, hence the one braket with the numbers of the corresponding alternatives [2,3,4] (walk is used as a base)
- travel time parameters for both classes are specified as alternative specific and included in the utilities of all modes, hence the one braket with the numbers of the corresponding alternatives [1,2,3,4]
- travel cost parameters for both classes are specified as generic and included only in the utilities of drive and pt, hence the two brakets and the numbers of the alternatives [[3,4]]


The specification of the two classes is as follows:
##### Latent Class 1:

$V_{walk} =        \beta_{tt, class1} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class1} + \beta_{tt, class1} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class1} + \beta_{tt, class1} * TravelTime_{pt} + \beta_{cost, class1} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class1} + \beta_{tt, class1} * TravelTime_{drive} + \beta_{cost, class1} * TravelCost_{drive} $

##### Latent Class 2:

$V_{walk} =        \beta_{tt, class2} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class2} + \beta_{tt, class2} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class2} + \beta_{tt, class2} * TravelTime_{pt} + \beta_{cost, class2} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class2} + \beta_{tt, class2} * TravelTime_{drive} + \beta_{cost, class2} * TravelCost_{drive} $

In [13]:
# scale travel time from hours to minutes
long_london['travel_time'] = long_london['travel_time'] * 60

# add a column of ones named'intercept'. This will be used later to define the Alternative-Specific Constants (ASCs).
long_london['intercept']=np.ones(long_london.shape[0])

In [14]:
# NOTE: Specification and variable names must be in lists of ordered dictionaries.
# class_specific_specs defines the variables names to be used in the specification of the 
# class specific choice model of each class.
# class_specific_labels defines the names associated with each of the variables which
# will be displayed in the output tables after estimation.


class_specific_specs = [OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])]),
                        OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])])]
                       
class_specific_labels = [OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])]),
                         OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])])]

In [15]:
# Starting Values for the Class-Specific Choice Model paramerters
paramClassSpec = []
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the first class
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the second class

### Step 5: Model Estimation and Results

To train/estimate the model, the following must be included in the GBM_LCCM.lccm_fit function:
- data: data used for the class-speficic choice model in long format
- X: continuous variables used for the class membership model in wide format
- X_Dummy: discrete variables used for the class membership model in wide format

For prediction/test, the following must be included in the GBM_LCCM.lccm_fit function:
- dataTest = data for the class-speficic choice model in long format
- XTest = continuous variables used for the class membership model in wide format
- XTest_Dummy = discrete variables used for the class membership model in wide format

Note that after training your model you can test it on a different dataset (test dataset). For this example, we use the same one that was used in training. However, you can split the data into train and test datasets(e.g. 80%-20%), estimate the model on the train dataset (data, X, X_Dummy) and test the model on the test dataset (dataTest, XTest, XTest_Dummy). The model will generate the prediction accuracy in terms of LL at the end of the notebook (Predicted Log-Likelihood).

The remaining variables that must be included in the GBM_LCCM.lccm_fit function are:
- prediction_test: 'Yes' if you would like to test the model on a test dataset and 'No' otherwise
- GMM_Initialization: 'random' to initialize the clusters of the Gaussian Bernoulli Mixture models randomly 
or 'kmeans' to initinalize the clusters using the k-means clustering algorithms
- covariance_type: the covariance structure of Gaussian Mixture Models can have four different types:
    - full, tied, diagonal and spherical. For more details about each type, you can refer to the following link: 
    - https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html
- ind_id_col: name of column identifying the individual for each row of data 
- obs_id_col: name of column identifying the observation (choice scenario)
- choice_col: name of column identifying whether the alternative corresponding by a row was chosen or not
- n_classes: number of latent classes (clusters)
- paramClassSpec: starting values for the class-specific choice models. If paramClassSpec is None, the model will generate the starting values randomly


In [16]:
# Define a location to store the resutls 
outputFilePath = '.../GBM_LCCM_Example/'

# In this example we will assume that eahc trip was done by different individual
# So, that is why ind_id_col is equal to 'trip_id'


# Train/Estimate the model
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    
    GBM_LCCM.lccm_fit(data = long_london,
                      X = Standardized_X_Mem_C,
                      X_Dummy = X_Mem_D,
                      dataTest = long_london,
                      XTest = Standardized_X_Mem_C,
                      XTest_Dummy = X_Mem_D,
                      prediction_test = 'Yes',
                      GMM_Initialization = 'random',
                      ind_id_col = 'trip_id', 
                      obs_id_col = 'custom_id',
                      alt_id_col = 'mode_id',
                      choice_col = 'travel_mode_nb', 
                      n_classes = n_classes,
                      covariance_type = 'full',
                      reg_covar = 1e-6,
                      tol = 1e-3,
                      max_iter = 100,
                      class_specific_specs = class_specific_specs,
                      class_specific_labels = class_specific_labels,
                      #paramClassSpec = paramClassSpec,
                      outputFilePath = outputFilePath,
                      outputFileName = 'ModelResults')

Processing data
Initializing EM Algorithm...

<Tue, 11 Feb 2025 11:50:03> Iteration 0: -70500.0190
<Tue, 11 Feb 2025 11:50:03> Iteration 1: -62858.7668
<Tue, 11 Feb 2025 11:50:03> Iteration 2: -62224.6450
<Tue, 11 Feb 2025 11:50:03> Iteration 3: -61543.7753
<Tue, 11 Feb 2025 11:50:03> Iteration 4: -61140.5397
<Tue, 11 Feb 2025 11:50:03> Iteration 5: -60997.2479
<Tue, 11 Feb 2025 11:50:04> Iteration 6: -60953.2884
<Tue, 11 Feb 2025 11:50:04> Iteration 7: -60936.9973
<Tue, 11 Feb 2025 11:50:04> Iteration 8: -60929.7982
<Tue, 11 Feb 2025 11:50:04> Iteration 9: -60926.4189
<Tue, 11 Feb 2025 11:50:04> Iteration 10: -60924.7391
<Tue, 11 Feb 2025 11:50:04> Iteration 11: -60923.8345
<Tue, 11 Feb 2025 11:50:04> Iteration 12: -60923.3024
<Tue, 11 Feb 2025 11:50:04> Iteration 13: -60922.9616
<Tue, 11 Feb 2025 11:50:04> Iteration 14: -60922.7266
<Tue, 11 Feb 2025 11:50:04> Iteration 15: -60922.5543
<Tue, 11 Feb 2025 11:50:05> Iteration 16: -60922.4218
<Tue, 11 Feb 2025 11:50:05> Iteration 17: -609